In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
%matplotlib inline

# Load MNIST

In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)
trainimg   = mnist.train.images
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print ("MNIST ready")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST ready


In [3]:
# NETWORK TOPOLOGIES
n_input    = 784
n_channel  = 64 
n_channel2 = 128
n_classes  = 10  

# INPUTS AND OUTPUTS
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
    
# NETWORK PARAMETERS
stddev = 0.1
w = {
    'c1': tf.Variable(tf.truncated_normal([3, 3, 1, n_channel], stddev=stddev)),
    'c2': tf.Variable(tf.truncated_normal([3, 3, n_channel, n_channel2], stddev=stddev)),
    'd1': tf.Variable(tf.truncated_normal([7*7*n_channel2, n_classes], stddev=stddev))
}
b = {
    'c1': tf.Variable(tf.random_normal([n_channel], stddev=stddev)),
    'c2': tf.Variable(tf.random_normal([n_channel2], stddev=stddev)),
    'd1': tf.Variable(tf.random_normal([n_classes], stddev=stddev))
}
print ("NETWORK READY")

NETWORK READY


In [4]:
def CNN(_input, _w, _b):
    _input_r = tf.reshape(_input, shape=[-1,28,28,1])
    #Layer1
    conv1 = tf.nn.conv2d(_input_r, _w['c1'], strides=[1,1,1,1], padding="SAME")
    _mean, _val = tf.nn.moments(conv1, [0,1,2])
    conv1 = tf.nn.bias_add(conv1, _b['c1'])
    conv1 = tf.nn.relu(conv1)
    _pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    
    #Layer2
    conv2 = tf.nn.conv2d(_pool1, _w['c2'], strides=[1,1,1,1], padding="SAME")
    conv2 = tf.nn.bias_add(conv2, _b['c2'])
    conv2 = tf.nn.relu(conv2)
    _pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    
    #fully connected
    _dense = tf.reshape(_pool2, [-1, 7*7*128])
    _out = tf.add(tf.matmul(_dense, _w['d1']), _b['d1'])
    
    out = {
        'out':_out
    }
    return out
    

In [5]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

learning_rate = 0.01
epoch_size = 5
batch_size = 100
show_step = 2
save_step = 1

pred = CNN(X, w, b)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optm = tf.train.AdamOptimizer(learning_rate).minimize(cost)
_cor = tf.equal(tf.arg_max(Y,1), tf.arg_max(pred, 1))
accr = tf.reduce_mean(tf.cast(_cor, tf.float32))
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

saver = tf.train.Saver(max_to_keep=3)

In [7]:
for epoch in range(epoch_size):
    total_batch = int(mnist.train.num_examples/batch_size)
    avg_cost = 0
    for step in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm, feed_dict={X:batch_xs, Y : batch_ys})
        avg_cost += sess.run(cost, feed_dict={X:batch_xs, Y : batch_ys})
        
    if epoch % show_step == 0:
        train_accr = sess.run(accr, feed_dict={X:batch_xs, Y : batch_ys})
        print("train accuracy : %.3f" % train_accr)
        test_accr = sess.run(accr, feed_dict={X:mnist.test.images, Y:mnist.test.labels})
        print("test accuracy : %.3f" % test_accr)
    if epoch % save_step == 0:
        saver.save(sess, 'nets/cnn_modern.ckpt-'+str(epoch))
print("finish training")

train accuracy : 0.970
test accuracy : 0.982
train accuracy : 0.980
test accuracy : 0.986
train accuracy : 0.990
test accuracy : 0.987
finish training
